# Blood vessel detection using UNET

First we divide images into adequate folders, segmenting train and test images to 128x128 windows:

In [1]:
from skimage.util import view_as_windows
from random import sample
import cv2
import shutil 
import os
import glob

TRAIN_IMG_DIR = 'data/train_images'
VAL_IMG_DIR = 'data/val_images'
TEST_IMG_DIR = 'data/test_images'

TRAIN_MASK_DIR = 'data/train_masks'
VAL_MASK_DIR = 'data/val_masks'
TEST_MASK_DIR = 'data/test_masks'

if not os.path.exists(TRAIN_IMG_DIR):
    os.makedirs(TRAIN_IMG_DIR)

if not os.path.exists(VAL_IMG_DIR):
    os.makedirs(VAL_IMG_DIR)

if not os.path.exists(TEST_IMG_DIR):
    os.makedirs(TEST_IMG_DIR)
    
if not os.path.exists(TRAIN_MASK_DIR):
    os.makedirs(TRAIN_MASK_DIR)

if not os.path.exists(VAL_MASK_DIR):
    os.makedirs(VAL_MASK_DIR)

if not os.path.exists(TEST_MASK_DIR):
    os.makedirs(TEST_MASK_DIR)
    
    
img_filenames = glob.glob("data/images/*")
img_filenames.sort()
images = [cv2.imread(img) for img in img_filenames]

mask_filenames = glob.glob("data/manual1/*")
mask_filenames.sort()
masks = [cv2.imread(img, 0) for img in mask_filenames]

# copy test images
for i, (img_path, mask_path) in enumerate(zip(img_filenames[:5], mask_filenames[:5])):
    shutil.copy(img_path, TEST_IMG_DIR + "/" + str(i) + ".png")
    shutil.copy(mask_path, TEST_MASK_DIR + "/" + str(i) + ".png")

# create windows for train and val images
all_windows = []
all_mask_windows = []
for img, mask in zip(images[5:], masks[5:]):
    windows = view_as_windows(img, (128, 128, 3), step=128)
    width, height, _, _, _, _ = windows.shape
    windows = windows.flatten().reshape(width*height, 128, 128, 3)
    all_windows.extend(windows)
    
    mask_windows = view_as_windows(mask, (128, 128), step=128)
    mask_windows = mask_windows.flatten().reshape(width*height, 128, 128)
    all_mask_windows.extend(mask_windows)

data = list(zip(all_windows, all_mask_windows))
after_sampling = sample(data, 100)
        
all_windows, all_mask_windows = zip(*after_sampling)

for i, (window, mask_window) in enumerate(zip(all_windows, all_mask_windows)):
    if i <= len(all_windows) * 0.8: # add to train set
        cv2.imwrite(TRAIN_IMG_DIR + "/" + str(i) + ".png", window)
        cv2.imwrite(TRAIN_MASK_DIR + "/" + str(i) + ".tiff", mask_window)
    else: # add to val set
        cv2.imwrite(VAL_IMG_DIR + "/" + str(i) + ".png", window)
        cv2.imwrite(VAL_MASK_DIR + "/" + str(i) + ".tiff", mask_window)

KeyboardInterrupt: 

Next we define UNET architecture:

In [27]:
import torch
import torch.nn as nn
import torchvision.transforms.functional as TF

class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        return self.conv(x)

class UNET(nn.Module):
    def __init__(
            self, in_channels=3, out_channels=1, features=[64, 128, 256, 512],
    ):
        super(UNET, self).__init__()
        self.ups = nn.ModuleList()
        self.downs = nn.ModuleList()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Down part of UNET
        for feature in features:
            self.downs.append(DoubleConv(in_channels, feature))
            in_channels = feature

        # Up part of UNET
        for feature in reversed(features):
            self.ups.append(
                nn.ConvTranspose2d(
                    feature*2, feature, kernel_size=2, stride=2,
                )
            )
            self.ups.append(DoubleConv(feature*2, feature))

        self.bottleneck = DoubleConv(features[-1], features[-1]*2)
        self.final_conv = nn.Conv2d(features[0], out_channels, kernel_size=1)

    def forward(self, x):
        skip_connections = []

        for down in self.downs:
            x = down(x)
            skip_connections.append(x)
            x = self.pool(x)

        x = self.bottleneck(x)
        skip_connections = skip_connections[::-1]

        for idx in range(0, len(self.ups), 2):
            x = self.ups[idx](x)
            skip_connection = skip_connections[idx//2]

            if x.shape != skip_connection.shape:
                x = TF.resize(x, size=skip_connection.shape[2:])

            concat_skip = torch.cat((skip_connection, x), dim=1)
            x = self.ups[idx+1](concat_skip)

        return self.final_conv(x)

Then we create dataset

In [28]:
from torch.utils.data import Dataset

class BloodVesselDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.images = glob.glob(image_dir + "/*")
        self.images.sort()
        self.masks = glob.glob(mask_dir + "*/.tif")
        self.masks.sort()
        
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, index):
        img_path = self.images[index]
        mask_path = self.images[index]
        image = cv2.imread(img_path)
        mask = cv2.imread(mask_path, 0)
        mask[mask == 255] = 1
        
        if self.transform is not None:
            augmentations = self.transform(image=image, mask=mask)
            image = augmentations["image"]
            mask = augmentations["mask"]
        
        return image, mask

In [36]:
import torch
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm
import torch.optim as optim

# HYPERPARAMETERS
LEARNING_RATE = 1e-4
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 16
NUM_EPOCHS = 3
NUM_WORKERS = 0
IMAGE_HEIGHT =  128
IMAGE_WIDTH = 128 
PIN_MEMORY = True
LOAD_MODEL = True


def train(loader, model, optimizer, loss_fn, scaler):
    loop = tqdm(loader)
    
    for batch_idx, (data, targets) in enumerate(loop):
        data = data.to(device=DEVICE)
        targets = targets.float().unsqueeze(1).to(device=DEVICE)
        
        # forward
        with torch.cuda.amp.autocast():
            predictions = model(data)
            loss = loss_fn(predictions, targets)
            
        # backward
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        # update tqdm loop
        loop.set_postfix(loss=loss.item())

        
train_transform = A.Compose(
[
    A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
    A.Rotate(limit=35, p=1.0),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.1),
    A.Normalize(mean = [0.0, 0.0, 0.0], std = [1.0, 1.0, 1.0], max_pixel_value = 255.0),
    ToTensorV2()
])

val_transform = A.Compose(
[
    A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
    A.Normalize(mean = [0.0, 0.0, 0.0], std = [1.0, 1.0, 1.0], max_pixel_value = 255.0),
    ToTensorV2()
])


In [30]:
import torchvision
from torch.utils.data import DataLoader

def save_checkpoint(state, filename):
    print("===>Saving checkpoint")
    torch.save(state, filename)
    
def load_checkpoint(checkpoint, model):
    print("===>Loading checkpoint")
    model.load_state_dict(checkpoint["state_dict"])
    
def get_loaders(
        train_dir,
        train_mask_dir,
        val_dir,
        val_mask_dir,
        batch_size,
        train_transform,
        val_transform,
        num_workers=4,
        pin_memory=True):
    
    train_ds = BloodVesselDataset(
        image_dir=train_dir,
        mask_dir=train_mask_dir,
        transform=train_transform)
    
    train_loader = DataLoader(
        train_ds,
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=pin_memory,
        shuffle=True)
    
    val_ds = BloodVesselDataset(
        image_dir=val_dir,
        mask_dir=val_mask_dir,
        transform=val_transform)
    
    val_loader = DataLoader(
        val_ds,
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=pin_memory,
        shuffle=False)
    
    return train_loader, val_loader

def check_accuracy(loader, model, device="cuda"):
    num_correct = 0
    num_pixels = 0
    dice_score = 0
    model.eval()
    
    with torch.no_grad():
        for x,y in loader:
            x = x.to(device)
            y = y.to(device)
            preds = torch.sigmoid(model(x))
            preds = (preds > 0.5).float()
            num_correct += (preds == y).sum()
            num_pixels += torch.numel(preds)
            dice_score = (2 * (preds * y).sum()) / ((preds + y).sum() + 1e-8)
    
    print(f"Got {num_correct}/{num_pixels} with acc {num_correct/num_pixels:.2f}")         
    print(f"Dice score: {dice_score/len(loader)}")
    model.train()

def save_predictions_as_imgs(
    loader, model, folder="saved_images/", device="cuda"
):
    model.eval()
    for idx, (x, y) in enumerate(loader):
        x = x.to(device=device)
        with torch.no_grad():
            preds = torch.sigmoid(model(x))
            preds = (preds > 0.5).float()
        torchvision.utils.save_image(
            preds, f"{folder}pred_{idx}.png"
        )
        #torchvision.utils.save_image(y.long().unsqueeze(1), f"{folder}mask_{idx}.png")

    model.train()

In [37]:
if __name__ == '__main__':
    model = UNET(in_channels=3, out_channels=1).to(DEVICE)
    loss_fn = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

    train_loader, val_loader = get_loaders(
        TRAIN_IMG_DIR,
        TRAIN_MASK_DIR,
        VAL_IMG_DIR,
        VAL_MASK_DIR,
        BATCH_SIZE,
        train_transform,
        val_transform,
        NUM_WORKERS,
        PIN_MEMORY
    )

    scaler = torch.cuda.amp.GradScaler()

    for epoch in range(NUM_EPOCHS):
        train(train_loader, model, optimizer, loss_fn, scaler)
        checkpoint = {
            "state_dict": model.state_dict(),
            "optimizer": optimizer.state_dict()
        }
        save_checkpoint(checkpoint, filename=f"checkpoint_{epoch}")
        check_accuracy(val_loader, model, device=DEVICE)
        save_predictions_as_imgs(val_loader, model, device=DEVICE)

100%|█████████████████████████████████████████████████████████████████████████| 6/6 [01:36<00:00, 16.09s/it, loss=22.1]


===>Saving checkpoint
Got 1085072/311296 with acc 3.49
Dice score: 0.0


100%|█████████████████████████████████████████████████████████████████████████| 6/6 [01:36<00:00, 16.09s/it, loss=11.4]


===>Saving checkpoint
Got 885640/311296 with acc 2.85
Dice score: 0.19999247789382935


100%|█████████████████████████████████████████████████████████████████████████| 6/6 [01:35<00:00, 15.92s/it, loss=10.7]


===>Saving checkpoint
Got 619045/311296 with acc 1.99
Dice score: 0.3820360600948334


## Bibliography:
https://www.youtube.com/watch?v=IHq1t7NxS8k&ab_channel=AladdinPersson